# 03_IDAO_Preproc_Train_Interpolate

* поминутная интерполяция значений обучающего датасета

In [1]:
import sys
import warnings
warnings.filterwarnings('ignore')

import math
from datetime import datetime, timedelta

import matplotlib.pylab as plt
import pandas as pd
import numpy as np

from scipy import interpolate

In [2]:
#Тренировочные данные
df_train = pd.read_csv('data/train_proc_with_id.csv', index_col='datetime', parse_dates=True)
#Тестовые данные
df_test = pd.read_csv('data/Track 1/test_proc_with_id.csv', index_col='datetime', parse_dates=True)

# Задача 1
submission = pd.read_csv('data/Track 1/submission.csv')

df_train = df_train.drop(['Unnamed: 0'], axis=1)
df_test = df_test.drop(['Unnamed: 0'], axis=1)

In [3]:
df_train.head(3)

,id,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
datetime,,,,,,,,,,,,,,
2014-01-01 00:00:00,0,0,-8855.823863,13117.780146,-20728.353233,-0.908303,-3.808436,-2.022083,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133
2014-01-01 00:46:43,1,0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468
2014-01-01 01:33:26,2,0,-10578.684043,-10180.467460,-24238.280949,0.277435,-4.047522,0.723155,-10571.858472,-10145.939908,-24271.169776,0.274880,-4.046788,0.718768


Добавим столбец, переводящий время в абсолютное значение:

In [4]:
df_train = df_train.reset_index()
df_train['time'] = (df_train['datetime'] - np.datetime64('1970-01-01 00:00:00')) / np.timedelta64(1, 's')

Вычислим индексы с совпадающими метками времени для дальнейшей обработки:

In [5]:
equal_values = []
for i in range(df_train.shape[0] - 1):
    time_prv = list(df_train.loc[i, ['datetime']])
    time_now = list(df_train.loc[i + 1, ['datetime']])
    if time_prv == time_now:
        equal_values.append(i)

Исправим совпадающие временные метки при помощи уравнения равноускоренного движения:

In [24]:
def improve_time_equals(df, equals_list):
    amount = len(equals_list)
    count = 0
    
    for id0 in equals_list:
        id1 = id0 + 1
        id2 = id0 + 2
        
        x0 = float(df.loc[id0, ['x_sim']])
        x1 = float(df.loc[id1, ['x_sim']])
        x2 = float(df.loc[id2, ['x_sim']])

        S02 = x2 - x0

        V0 = float(df.loc[id0, ['Vx_sim']])
        V1 = float(df.loc[id1, ['Vx_sim']])
        V2 = float(df.loc[id2, ['Vx_sim']])

        t0 = float(df.loc[id0, ['time']])
        t2 = float(df.loc[id2, ['time']])
        
        print(df.loc[id0, ['datetime']])
        print(df.loc[id1, ['datetime']])
        print(df.loc[id2, ['datetime']])
        
        T02 = t2 - t0
        
        a = 2 * (S02 - V0 * T02) / T02**2
        
        A = a / 2
        B = V0
        C = x0 - x1

        discr = B ** 2 - 4 * A * C
        if discr >= 0:
            count += 1
            print(count, '/', amount)
            t1_1 = int((-B - math.sqrt(discr)) / (2 * A))
            t1_2 = int((-B + math.sqrt(discr)) / (2 * A))
            print('Корни уравнения:', t1_1, 'и', t1_2)
            if min(t1_1, t1_2) < 0:
                t1 = max(t1_1, t1_2)
            else:
                t1 = min(t1_1, t1_2)
            print('Секунд:', t1)
            time_delta = timedelta(seconds=t1)
            df.loc[id1, ['datetime']] = df.loc[id1, ['datetime']] + time_delta
            
        else:
            print('D < 0 в строке:', id0)
        
        print(df.loc[id0, ['datetime']])
        print(df.loc[id1, ['datetime']])
        print(df.loc[id2, ['datetime']])
        print()
        
    return df

In [7]:
df_train_improved = improve_time_equals(df_train, equal_values)

datetime    2014-01-09 15:06:38
Name: 266, dtype: object
datetime    2014-01-09 15:06:38
Name: 267, dtype: object
datetime    2014-01-09 15:53:21
Name: 268, dtype: object
1 / 496
Корни уравнения: -2277 и 1586
Секунд: 1586
datetime    2014-01-09 15:06:38
Name: 266, dtype: object
datetime    2014-01-09 15:33:04
Name: 267, dtype: object
datetime    2014-01-09 15:53:21
Name: 268, dtype: object

datetime    2014-01-26 21:19:54
Name: 799, dtype: object
datetime    2014-01-26 21:19:54
Name: 800, dtype: object
datetime    2014-01-26 22:06:37
Name: 801, dtype: object
2 / 496
Корни уравнения: -4962 и 1761
Секунд: 1761
datetime    2014-01-26 21:19:54
Name: 799, dtype: object
datetime    2014-01-26 21:49:15
Name: 800, dtype: object
datetime    2014-01-26 22:06:37
Name: 801, dtype: object

datetime    2014-01-12 08:23:12
Name: 1729, dtype: object
datetime    2014-01-12 08:23:12
Name: 1730, dtype: object
datetime    2014-01-12 08:44:24
Name: 1731, dtype: object
3 / 496
Корни уравнения: 793 и -2065
С

datetime    2014-01-26 15:57:23
Name: 21833, dtype: object
datetime    2014-01-26 15:57:23
Name: 21834, dtype: object
datetime    2014-01-26 16:58:59
Name: 21835, dtype: object
24 / 496
Корни уравнения: 2334 и -5630
Секунд: 2334
datetime    2014-01-26 15:57:23
Name: 21833, dtype: object
datetime    2014-01-26 16:36:17
Name: 21834, dtype: object
datetime    2014-01-26 16:58:59
Name: 21835, dtype: object

datetime    2014-01-08 13:37:31
Name: 22331, dtype: object
datetime    2014-01-08 13:37:31
Name: 22332, dtype: object
datetime    2014-01-08 14:06:49
Name: 22333, dtype: object
25 / 496
Корни уравнения: 1124 и -3583
Секунд: 1124
datetime    2014-01-08 13:37:31
Name: 22331, dtype: object
datetime    2014-01-08 13:56:15
Name: 22332, dtype: object
datetime    2014-01-08 14:06:49
Name: 22333, dtype: object

datetime    2014-01-16 03:15:03
Name: 22704, dtype: object
datetime    2014-01-16 03:15:03
Name: 22705, dtype: object
datetime    2014-01-16 03:44:20
Name: 22706, dtype: object
D < 0 в с

Name: 41958, dtype: object
datetime    2014-01-25 06:21:02
Name: 41959, dtype: object

datetime    2014-01-09 12:34:06
Name: 47275, dtype: object
datetime    2014-01-09 12:34:06
Name: 47276, dtype: object
datetime    2014-01-09 14:44:41
Name: 47277, dtype: object
48 / 496
Корни уравнения: -22131 и 5651
Секунд: 5651
datetime    2014-01-09 12:34:06
Name: 47275, dtype: object
datetime    2014-01-09 14:08:17
Name: 47276, dtype: object
datetime    2014-01-09 14:44:41
Name: 47277, dtype: object

datetime    2014-01-18 01:08:13
Name: 47370, dtype: object
datetime    2014-01-18 01:08:13
Name: 47371, dtype: object
datetime    2014-01-18 03:18:48
Name: 47372, dtype: object
49 / 496
Корни уравнения: -11934 и 4879
Секунд: 4879
datetime    2014-01-18 01:08:13
Name: 47370, dtype: object
datetime    2014-01-18 02:29:32
Name: 47371, dtype: object
datetime    2014-01-18 03:18:48
Name: 47372, dtype: object

datetime    2014-01-26 13:42:20
Name: 47465, dtype: object
datetime    2014-01-26 13:42:20
Name: 

Name: 84622, dtype: object
71 / 496
Корни уравнения: -1852 и 693
Секунд: 693
datetime    2014-01-26 12:15:55
Name: 84620, dtype: object
datetime    2014-01-26 12:27:28
Name: 84621, dtype: object
datetime    2014-01-26 12:34:31
Name: 84622, dtype: object

datetime    2014-01-07 12:47:45
Name: 87423, dtype: object
datetime    2014-01-07 12:47:45
Name: 87424, dtype: object
datetime    2014-01-07 15:14:44
Name: 87425, dtype: object
72 / 496
Корни уравнения: 58 и 2691
Секунд: 58
datetime    2014-01-07 12:47:45
Name: 87423, dtype: object
datetime    2014-01-07 12:48:43
Name: 87424, dtype: object
datetime    2014-01-07 15:14:44
Name: 87425, dtype: object

datetime    2014-01-14 01:35:30
Name: 87488, dtype: object
datetime    2014-01-14 01:35:30
Name: 87489, dtype: object
datetime    2014-01-14 04:02:29
Name: 87490, dtype: object
73 / 496
Корни уравнения: 5556 и -16141
Секунд: 5556
datetime    2014-01-14 01:35:30
Name: 87488, dtype: object
datetime    2014-01-14 03:08:06
Name: 87489, dtype: ob

Name: 98660, dtype: object
datetime    2014-01-12 19:11:47
Name: 98661, dtype: object

datetime    2014-01-18 14:16:51
Name: 98744, dtype: object
datetime    2014-01-18 14:16:51
Name: 98745, dtype: object
datetime    2014-01-18 15:57:24
Name: 98746, dtype: object
94 / 496
Корни уравнения: 3144 и -3832
Секунд: 3144
datetime    2014-01-18 14:16:51
Name: 98744, dtype: object
datetime    2014-01-18 15:09:15
Name: 98745, dtype: object
datetime    2014-01-18 15:57:24
Name: 98746, dtype: object

datetime    2014-01-24 11:02:28
Name: 98829, dtype: object
datetime    2014-01-24 11:02:28
Name: 98830, dtype: object
datetime    2014-01-24 12:43:01
Name: 98831, dtype: object
95 / 496
Корни уравнения: -9391 и 3748
Секунд: 3748
datetime    2014-01-24 11:02:28
Name: 98829, dtype: object
datetime    2014-01-24 12:04:56
Name: 98830, dtype: object
datetime    2014-01-24 12:43:01
Name: 98831, dtype: object

datetime    2014-01-16 03:19:20
Name: 99529, dtype: object
datetime    2014-01-16 03:19:20
Name: 99

...

datetime    2014-01-19 04:52:10
Name: 535009, dtype: object
datetime    2014-01-19 04:52:10
Name: 535010, dtype: object
datetime    2014-01-19 05:17:34
Name: 535011, dtype: object
358 / 496
Корни уравнения: 925 и -2041
Секунд: 925
datetime    2014-01-19 04:52:10
Name: 535009, dtype: object
datetime    2014-01-19 05:07:35
Name: 535010, dtype: object
datetime    2014-01-19 05:17:34
Name: 535011, dtype: object

datetime    2014-01-06 17:41:20
Name: 537399, dtype: object
datetime    2014-01-06 17:41:20
Name: 537400, dtype: object
datetime    2014-01-06 20:49:05
Name: 537401, dtype: object
359 / 496
Корни уравнения: -16994 и 6918
Секунд: 6918
datetime    2014-01-06 17:41:20
Name: 537399, dtype: object
datetime    2014-01-06 19:36:38
Name: 537400, dtype: object
datetime    2014-01-06 20:49:05
Name: 537401, dtype: object

datetime    2014-01-15 08:13:20
Name: 537466, dtype: object
datetime    2014-01-15 08:13:20
Name: 537467, dtype: object
datetime    2014-01-15 11:21:05
Name: 537468, dtype: 

Name: 556872, dtype: object

datetime    2014-01-10 03:06:37
Name: 558136, dtype: object
datetime    2014-01-10 03:06:37
Name: 558137, dtype: object
datetime    2014-01-10 04:46:13
Name: 558138, dtype: object
382 / 496
Корни уравнения: 3811 и -11889
Секунд: 3811
datetime    2014-01-10 03:06:37
Name: 558136, dtype: object
datetime    2014-01-10 04:10:08
Name: 558137, dtype: object
datetime    2014-01-10 04:46:13
Name: 558138, dtype: object

datetime    2014-01-19 06:13:15
Name: 558269, dtype: object
datetime    2014-01-19 06:13:15
Name: 558270, dtype: object
datetime    2014-01-19 07:52:51
Name: 558271, dtype: object
D < 0 в строке: 558269
datetime    2014-01-19 06:13:15
Name: 558269, dtype: object
datetime    2014-01-19 06:13:15
Name: 558270, dtype: object
datetime    2014-01-19 07:52:51
Name: 558271, dtype: object

datetime    2014-01-09 11:17:40
Name: 563326, dtype: object
datetime    2014-01-09 11:17:40
Name: 563327, dtype: object
datetime    2014-01-09 17:38:51
Name: 563328, dtype:

Name: 587954, dtype: object
datetime    2014-01-17 22:50:32
Name: 587955, dtype: object
datetime    2014-01-17 23:41:47
Name: 587956, dtype: object

datetime    2014-01-06 21:35:39
Name: 589010, dtype: object
datetime    2014-01-06 21:35:39
Name: 589011, dtype: object
datetime    2014-01-06 23:33:39
Name: 589012, dtype: object
404 / 496
Корни уравнения: 4950 и -21484
Секунд: 4950
datetime    2014-01-06 21:35:39
Name: 589010, dtype: object
datetime    2014-01-06 22:58:09
Name: 589011, dtype: object
datetime    2014-01-06 23:33:39
Name: 589012, dtype: object

datetime    2014-01-12 19:11:19
Name: 589083, dtype: object
datetime    2014-01-12 19:11:19
Name: 589084, dtype: object
datetime    2014-01-12 21:09:19
Name: 589085, dtype: object
405 / 496
Корни уравнения: 4809 и -59419
Секунд: 4809
datetime    2014-01-12 19:11:19
Name: 589083, dtype: object
datetime    2014-01-12 20:31:28
Name: 589084, dtype: object
datetime    2014-01-12 21:09:19
Name: 589085, dtype: object

datetime    2014-01-2

Name: 601984, dtype: object
datetime    2014-01-20 05:24:56
Name: 601985, dtype: object
426 / 496
Корни уравнения: -1522 и 367
Секунд: 367
datetime    2014-01-20 05:15:39
Name: 601983, dtype: object
datetime    2014-01-20 05:21:46
Name: 601984, dtype: object
datetime    2014-01-20 05:24:56
Name: 601985, dtype: object

datetime    2014-01-29 19:53:29
Name: 603475, dtype: object
datetime    2014-01-29 19:53:29
Name: 603476, dtype: object
datetime    2014-01-29 20:02:45
Name: 603477, dtype: object
427 / 496
Корни уравнения: 293 и -358
Секунд: 293
datetime    2014-01-29 19:53:29
Name: 603475, dtype: object
datetime    2014-01-29 19:58:22
Name: 603476, dtype: object
datetime    2014-01-29 20:02:45
Name: 603477, dtype: object

datetime    2014-01-07 21:23:41
Name: 603867, dtype: object
datetime    2014-01-07 21:23:41
Name: 603868, dtype: object
datetime    2014-01-08 00:27:28
Name: 603869, dtype: object
428 / 496
Корни уравнения: 6693 и -15295
Секунд: 6693
datetime    2014-01-07 21:23:41
Nam

Name: 636163, dtype: object
datetime    2014-01-16 05:18:02
Name: 636164, dtype: object

datetime    2014-01-25 08:14:15
Name: 637603, dtype: object
datetime    2014-01-25 08:14:15
Name: 637604, dtype: object
datetime    2014-01-25 08:23:23
Name: 637605, dtype: object
D < 0 в строке: 637603
datetime    2014-01-25 08:14:15
Name: 637603, dtype: object
datetime    2014-01-25 08:14:15
Name: 637604, dtype: object
datetime    2014-01-25 08:23:23
Name: 637605, dtype: object

datetime    2014-01-11 00:28:52
Name: 641691, dtype: object
datetime    2014-01-11 00:28:52
Name: 641692, dtype: object
datetime    2014-01-11 01:45:37
Name: 641693, dtype: object
451 / 496
Корни уравнения: 2848 и -7016
Секунд: 2848
datetime    2014-01-11 00:28:52
Name: 641691, dtype: object
datetime    2014-01-11 01:16:20
Name: 641692, dtype: object
datetime    2014-01-11 01:45:37
Name: 641693, dtype: object

datetime    2014-01-26 01:12:10
Name: 641974, dtype: object
datetime    2014-01-26 01:12:10
Name: 641975, dtype: 

Не все значения удалось исправить. Получим список неисправленных значений:

In [9]:
equal_values = []
for i in range(df_train_improved.shape[0] - 1):
    time_prv = list(df_train_improved.loc[i, ['datetime']])
    time_now = list(df_train_improved.loc[i + 1, ['datetime']])
    if time_prv == time_now:
        equal_values.append(i)

Исправим оставшиеся значения. В качестве метки времени выберем момент времени, равноудалённый от соседних точек.

In [10]:
def improve_time_equals_last(df, equals_list):
    for id0 in equals_list:
        id1 = id0 + 1
        id2 = id0 + 2
        time0 = df.loc[id0, ['time']]
        time2 = df.loc[id0 + 2, ['time']]
        delta = int((time2 - time0)/2)
        time_delta = timedelta(seconds=delta)
        df.loc[id1, ['datetime']] = df.loc[id1, ['datetime']] + time_delta
    return df

In [11]:
df_train_improved = improve_time_equals_last(df_train_improved, equal_values)

Получим список спутников из тестовой выборки:

In [13]:
set_test_satellites_id = set(df_test['sat_id'])

Функции, обеспечивающие интерполяцию значений:

In [14]:
# производит разметку по времени в выборке значений одного спутника
def resample_for_interpolate(df_one_sat):
    df_one_sat['date'] = df_one_sat.index
    df_one_sat['time'] = (df_one_sat['date'] - np.datetime64('1970-01-01 00:00:00')) / np.timedelta64(1, 's')
    df_one_sat = df_one_sat.resample('min').mean()
    df_one_sat['date'] = df_one_sat.index
    df_one_sat['time'] = (df_one_sat['date'] - np.datetime64('1970-01-01 00:00:00')) / np.timedelta64(1, 's')
    return df_one_sat

In [15]:
# интерполирует одну колонку датафрейма
def interpolate_col_df(df_one_sat, df_one_sat_resampled, column='x'):
    x = df_one_sat['time']
    y = df_one_sat[column]
    xnew = df_one_sat_resampled['time']
    f = interpolate.interp1d(x, y, fill_value="extrapolate")
    ynew = f(xnew)
    df_one_sat_resampled[column] = ynew
    return df_one_sat_resampled

In [22]:
# интерполирует датафрейм целиком
def df_interpolate(df):
    df_result = pd.DataFrame(columns=['x', 'y', 'z', 'Vx', 'Vy', 'Vz', 'sat_id',
                                     'id', 'x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim',
                                     'time', 'date'])
    
    for sat in list(set_test_satellites_id):
    
        df_one_sat = df.query('sat_id == @sat')
        df_one_sat_resampled = resample_for_interpolate(df_one_sat)

        df_interpolated = interpolate_col_df(df_one_sat, df_one_sat_resampled, 'x')
        df_interpolated = interpolate_col_df(df_one_sat, df_one_sat_resampled, 'y')
        df_interpolated = interpolate_col_df(df_one_sat, df_one_sat_resampled, 'z')
        df_interpolated = interpolate_col_df(df_one_sat, df_one_sat_resampled, 'Vx')
        df_interpolated = interpolate_col_df(df_one_sat, df_one_sat_resampled, 'Vy')
        df_interpolated = interpolate_col_df(df_one_sat, df_one_sat_resampled, 'Vz')
        df_interpolated = interpolate_col_df(df_one_sat, df_one_sat_resampled, 'sat_id')
        df_interpolated = interpolate_col_df(df_one_sat, df_one_sat_resampled, 'x_sim')
        df_interpolated = interpolate_col_df(df_one_sat, df_one_sat_resampled, 'y_sim')
        df_interpolated = interpolate_col_df(df_one_sat, df_one_sat_resampled, 'z_sim')
        df_interpolated = interpolate_col_df(df_one_sat, df_one_sat_resampled, 'Vx_sim')
        df_interpolated = interpolate_col_df(df_one_sat, df_one_sat_resampled, 'Vy_sim')
        df_interpolated = interpolate_col_df(df_one_sat, df_one_sat_resampled, 'Vz_sim')
        
        df_result = pd.concat([df_result, df_interpolated])
    
    return df_result

Вернём датасет к исходному виду:

In [18]:
df_train_improved.index = df_train_improved['datetime']
df_train_improved = df_train_improved.drop(['datetime', 'time'], axis=1)

Применим интерполяцию к обучающему датасету:

In [23]:
df_final = df_interpolate(df_train_improved)

Проверим полученный датасет:

In [25]:
df_final.head(3)

,Vx,Vx_sim,Vy,Vy_sim,Vz,Vz_sim,date,id,sat_id,time,x,x_sim,y,y_sim,z,z_sim
2014-01-01 00:00:00,2.520477,2.508879,6.149930,6.152996,-2.827599,-2.826227,2014-01-01 00:00:00,1819.0,1.0,1.388534e+09,10390.313089,10405.813755,-2796.458271,-2771.180760,3179.562085,3166.926302
2014-01-01 00:01:00,2.348307,2.337052,6.134658,6.137385,-2.847816,-2.846235,2014-01-01 00:01:00,NaN,1.0,1.388534e+09,10428.328529,10443.297921,-2424.698448,-2399.492986,2989.854516,2977.491684
2014-01-01 00:02:00,2.176136,2.165225,6.119386,6.121774,-2.868033,-2.866242,2014-01-01 00:02:00,NaN,1.0,1.388535e+09,10466.343970,10480.782087,-2052.938626,-2027.805212,2800.146946,2788.057066


In [26]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13378994 entries, 2014-01-01 00:00:00 to 2014-01-31 23:56:00
Data columns (total 16 columns):
Vx        float64
Vx_sim    float64
Vy        float64
Vy_sim    float64
Vz        float64
Vz_sim    float64
date      datetime64[ns]
id        float64
sat_id    float64
time      float64
x         float64
x_sim     float64
y         float64
y_sim     float64
z         float64
z_sim     float64
dtypes: datetime64[ns](1), float64(15)
memory usage: 1.7 GB


Сохраним в файл:

In [27]:
df_final.to_csv('data/Track 1/train_final.csv')